In [1]:
import cv2
import numpy as np
from scipy import ndimage as ndi
from skimage.morphology import square,dilation
from skimage.io import imshow
import time
import copy

In [30]:

class HoughTransform:    
    def __init__(self,threshold = 30):
        self.url = '../video/my_video2.h264'
        self.cap = cv2.VideoCapture(self.url)
        self.maskR = False
        self.ret = 0
        self.frame = 0
        self.slope_right = 0
        self.slope_left = 0
        self.slope = 0
        self.font = cv2.FONT_HERSHEY_SIMPLEX
        self.threshold = threshold
    
    def region_of_interest(self,img, vertices):   
        mask = np.zeros_like(img)
        match_mask_color = (255,)
        cv2.fillPoly(mask, vertices, 255)
        masked_image = cv2.bitwise_and(img, mask)
        return masked_image
    
    def findSlope(self,start,end,height):
        x1 = start[0]
        x2 = end[0]
        y1 = height-start[1]
        y2 = height-end[1]
        if (x2 - x1) != 0:
            m = (y2-y1)/(x2-x1)
            return m 

    def avgSlope(self,li):             
        x1 = round(sum(li[:,0])/len(li)).astype('int')
        x2 = round(sum(li[:,1])/len(li)).astype('int')
        y1 = round(sum(li[:,2])/len(li)).astype('int')
        y2 = round(sum(li[:,3])/len(li)).astype('int')
        return x1,x2,y1,y2

    def debug(self,debug = False):
        if (debug):            
            #cv2.putText(self.frame,str(self.slope_right),(100,25), self.font, 0.5,(255,255,255),2,cv2.LINE_AA)
            print('slope_right :', self.slope_right)            
            print('slope_left :', self.slope_left)            
            print('slope :', self.slope)
            print()
            
    def run(self,frame=None,fromvideo=False,debugs = False):
        if(fromvideo):
            while self.cap.isOpened():
                self.ret, self.frame = self.cap.read()
                if not self.ret:
                    print("Can't receive frame (stream end?). Exiting ...")
                    break                
                if cv2.waitKey(1) == ord('q'):
                    break
                self.corerun()                    
                self.debug(debug = debugs)
            self.cap.release()
            cv2.destroyAllWindows()
        else:
            self.frame = frame
            while cv2.waitKey(1) != ord('q'):                
                self.corerun()  
                self.debug(debug = debugs)
                return float(self.slope),self.maskR
            self.cap.release()
            cv2.destroyAllWindows()
            
    def corerun(self):
        self.maskR = False
        self.frame = cv2.resize(self.frame,(200, 150)) 
        height=self.frame.shape[0]
        width=self.frame.shape[1]
        
        self.frame=cv2.cvtColor(self.frame, cv2.COLOR_BGR2RGB)    
        lowerR=np.array([150,100,100])
        upperR=np.array([255,190,190])    
        maskR = cv2.inRange(self.frame, lowerR, upperR)
        
        countR=0
        for i in range(width):            
            if maskR[30][i] == 255 or maskR[31][i] == 255 or maskR[32][i] == 255:
                countR+=1
#                 print(countR)
            if countR > 10:
                self.maskR = True
                # print('stop')
                break
        
        # color range
        lowerY=np.array([200,200,0])
        upperY=np.array([255,255,210])    
        maskY = cv2.inRange(self.frame, lowerY, upperY)

        lowerW=np.array([210,210,210])
        upperW=np.array([255,255,255])    
        maskW = cv2.inRange(self.frame, lowerW, upperW)

        
        # crop image
        region_of_interest_vertices = [   
            (0, height),
            (0, (height/3)+10),
            (width, (height/3)+10),
            (width, height)]
        cropW = self.region_of_interest(maskW,np.array([region_of_interest_vertices], np.int32))
        cropY = self.region_of_interest(maskY,np.array([region_of_interest_vertices], np.int32))
        
        cropW = dilation(cropW, square(3))
        cropY = dilation(cropY, square(5))
        
        linesW = cv2.HoughLinesP(cropW,rho = 1,theta = 1*np.pi/180,threshold = 40,minLineLength = 10,maxLineGap = 250)
        linesY = cv2.HoughLinesP(cropY,rho = 1,theta = 1*np.pi/180,threshold = 40,minLineLength = 10,maxLineGap = 250)

        l=[]
        if(linesW is not None): 
            l = np.round(np.array([sum(x)/len(x) for x in zip(*linesW)]))[0].astype('int')
            
        r=[]
        if(linesY is not None): 
            r = np.round(np.array([sum(x)/len(x) for x in zip(*linesY)]))[0].astype('int')
        dframe = copy.deepcopy(self.frame) 
        if(len(l)):                
            self.slope_left = self.findSlope((l[0],l[1]),(l[2],l[3]),height)
            cv2.line(dframe, (l[0],l[1]),(l[2],l[3]), (255, 0, 0), 3)
        else:
            self.slope_left = 0
            
        if(len(r)):                 
            self.slope_right = self.findSlope((r[0],r[1]),(r[2],r[3]),height)
            cv2.line(dframe, (r[0],r[1]),(r[2],r[3]), (0, 0, 255), 3)
        else:
            self.slope_right = 0

        self.slope = round(self.slope_right + self.slope_left,3)
        print('slope',self.slope)  
        if(self.slope > 2):
            self.slope = 2
        elif(self.slope < -2):
            self.slope = -2  
         
        cv2.putText(dframe,str(self.slope),(10,20), self.font, 0.5,(255,255,255),1,cv2.LINE_AA)

        cv2.imshow('frame', dframe)  

In [31]:
h = HoughTransform()
h.run(fromvideo=True)

slope 0
slope 0
slope 0
slope -1.0
slope -1.483
slope -1.038
slope -0.261
slope -0.514
slope -0.342
slope -0.112
slope -0.121
slope -0.595
slope -0.026
slope 0.044
slope -0.191
slope -0.519
slope 0.038
slope -0.325
slope -0.126
slope -0.367
slope -0.038
slope -0.319
slope -0.304
slope -0.04
slope -0.18
slope -0.036
slope -0.203
slope -0.264
slope -0.153
slope -0.201
slope -0.548
slope -0.214
slope -0.31
slope -0.232
slope -0.564
slope -0.374
slope -0.805
slope -0.184
slope -0.754
slope -0.278
slope -0.338
slope -0.926
slope -0.415
slope -0.708
slope -0.441
slope -0.679
slope -1.835
slope -1.104
slope -1.036
slope -0.871
slope -1.136
slope -1.6
slope -0.884
slope -0.947
slope -1.474
slope -1.278
slope -1.136
slope -1.764
slope -1.231
slope -1.101
slope -1.823
slope -1.6
slope -1.217
slope -1.2
slope -4.806
slope -4.762
slope -3.072
slope -4.367
slope -1.721
slope -2.497
slope -1.28
slope -3.8
slope -4.871
slope 9.667
slope 1.031
slope -2.143
slope -1.5
slope 22.033
slope -12.714
slope 1

TypeError: unsupported operand type(s) for +: 'float' and 'NoneType'